In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import re

In [2]:
def player_stats_by_year():

  all_players = []

  current_year = 1989
  end_year = 2020
  all_nba_team = all_nba()

  
  for year in range(current_year, end_year +1):
    # Load the yearly player stats
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)
    # Find the statistics/column names
    columns = []
    for col in soup.find('tr').findAll('th'):
      columns.append(col.getText())
    columns.append("YEAR")
    columns.append("ALL-NBA")
    columns = columns [1:]


    # Find the player data for each column
    players = soup.findAll('tr')[1:]
    for player in range(len(players)):
      stats = []
      for stat in players[player].findAll('td'):
        stats.append(stat.getText())
      stats.append(year)
      # Players that are inducted in the hall of fame have an 
      # asterisks beside their names. We need to remove it to
      # properly match with the All NBA data.
      if str(stats[0])[-1] == "*":
        stats[0] = stats[0][:-1]
      # If player is in an All NBA team assign them a 1, 
      # otherwise a 0.
      if year in all_nba_team and stats[0] in all_nba_team[year]:
        stats.append(1)
      else:
        stats.append(0)
      all_players.append(stats)


  stats = pd.DataFrame(all_players, columns = columns)
  return stats

In [3]:
def all_nba():
  
  url = "https://www.basketball-reference.com/awards/all_league.html"
  html = urlopen(url)
  soup = BeautifulSoup(html)

  all_nba_by_year = {}
  start_year = 1988
  end_year = 2020
  counter = 0
  
  # Find every player 
  for row in soup.find_all('tr'):
    players = row.find_all('a')[2:]
    for player in players:
      player = player.getText()
      if counter == 15:
        end_year -= 1
        counter = 0
      if end_year >= start_year:
        if end_year in all_nba_by_year:
          all_nba_by_year[end_year].append(player)
        else:
          all_nba_by_year[end_year] = [player]
      counter += 1


  return  all_nba_by_year

## Data Preprocessing


Lets look at a few years of player data, to see what we are working with.

In [ ]:
players_df = player_stats_by_year()

In [ ]:
players_df.head()

In [ ]:
print("The shape of the dataset is :", players_df.shape)

In [ ]:
players_df.groupby(['YEAR']).agg(['sum'])['ALL-NBA']

In [ ]:
players_df.dtypes

### Removing null values

In [ ]:
players_df.isnull().sum()

In [ ]:
players_df.dropna(axis=0, inplace=True)

### Removing duplicate values

Let's look at an example of a player who has been traded mid-season. 

In [ ]:
players_df[(players_df["Player"] == 'Robert Covington') & (players_df["YEAR"] == 2019)]

In [ ]:
players_df.drop_duplicates(subset=['Player', 'YEAR'], keep='first', inplace=True)

If we run the query on that specific player again, we will only see one row.

In [ ]:
players_df[(players_df["Player"] == 'Robert Covington') & (players_df["YEAR"] == 2019)]

### Changing player positions


In [ ]:
players_df["Pos"].unique()

Notice that player can be listed as having played in multiple positions. This can be attributed to the fact players switching teams mid-season may have a new role on the new team. The data lists the position played most first, so I will be using that as the main position. 

In [ ]:
def main_position(position):

  if position.startswith('C'):
    return 'C'
  elif position == 'G':
    return 'PG'
  else:
    return position[:2]

players_df['Pos'] = players_df['Pos'].apply(main_position)

We can see if this worked by checking the unique values again.

In [ ]:
players_df["Pos"].unique()

If we were to one hot encode the positions, we would come up with an error. This is because we already have an existing column PF(personal fouls) with the position PF(power foward). So we can simply fix this by changing the personal fouls column name to "FOULS".

In [ ]:
players_df.rename(columns={"PF": "FOULS"}, inplace=True )
one_hot = pd.get_dummies(players_df['Pos'])
players_df.drop('Pos', axis = 1, inplace=True)
players_df = players_df.join(one_hot)

In [ ]:
players_df.drop(['Tm', 'Player', '3P%'], axis=1, inplace=True)

In [ ]:
players_df = players_df.apply(pd.to_numeric)

In [ ]:
players_df.dtypes

In [ ]:
players_df = players_df[(players_df['G'] > 15) & (players_df['MP'] > 15)]

In [ ]:
players_df.shape

## Exploratory Data Analysis

First lets take a look at each individual statistic's distribution.


In [ ]:
width = 5
height = 6

fig, ax = plt.subplots(height, width)
fig.subplots_adjust(bottom = 0, top = height - 1, 
                    left = 0, right = width - 1 )

columns = players_df.columns[1:30]

n = 0
m = 0
for col in columns:
  sns.histplot(players_df[col], kde=True, element="step", ax = ax[n][m])
  if m == width - 1:
    m = 0
    n += 1
  else:
    m += 1

Next, we can compare the boxplots of the statistics between all NBA and non all NBA players.



In [ ]:
width = 3
height = 7

fig, ax = plt.subplots(height, width)
fig.subplots_adjust(bottom = 0, top = height - 1, 
                    left = 0, right = width - 1 )

columns = players_df.columns[2:23]

n = 0
m = 0
for col in columns:
  sns.boxplot(x="ALL-NBA", y = col, data=players_df, ax = ax[n][m])
  if m == width - 1:
    m = 0
    n += 1
  else:
    m += 1

Finally, we can check the correlation of each player statistic.

In [ ]:
corr = players_df.corr()
fig, ax = plt.subplots(figsize=(20, 20))
mask = np.triu(np.ones_like(corr, dtype=bool))

cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Creating train/test data
 

In [ ]:
predictors = players_df.drop('ALL-NBA', axis=1, inplace=False)
outcome = players_df['ALL-NBA']
X_train, X_test, Y_train, Y_test = train_test_split(predictors, outcome, test_size = 0.3)

In [ ]:
print("The shape of X train is :", X_train.shape)
print("The shape of X test is :", X_test.shape)
print("The shape of Y train is :", Y_train.shape)
print("The shape of Y test is :", Y_test.shape)

## Univariate logistic regression

In [ ]:
def logistic_regression(col, weight=None):

  X_train, X_test, Y_train, Y_test = train_test_split(predictors, outcome, test_size = 0.3)

  if type(col) == str:
    X_test = X_test[col].values.reshape(-1, 1)
    X_train = X_train[col].values.reshape(-1, 1)
  else:
    X_test = X_test[col]
    X_train = X_train[col]

  model = LogisticRegression(class_weight = weight)
  model.fit(X_train, Y_train)

  predict = model.predict(X_test)
  accuracy = model.score(X_test, Y_test)
  roc = roc_auc_score(Y_test, predict)

  print("Accuracy: {}".format(accuracy))
  print("AUC-ROC: {}".format(roc))

  return model

In [ ]:
logistic_regression('PTS', weight = 'balanced')

In [ ]:
logistic_regression('STL', weight = 'balanced')

In [ ]:
logistic_regression('FG%', weight = 'balanced')

Looking at the scores of these univarate logistic regression models, it seems like it's prediction is good. However, the evaluation metric that score() uses is the accuracy. This is the percentage of outcomes that the model has predicted correctly. However, since there are such a low number of all-nba players (~5%), the model can predict the players to be all false (not all nba team) and still achieve an accuracy of ~95%.

Alternatively we can look at the confusion matrix below, where the,
  - top left value: actual value is false and predicted value is false (True negative)
  - top right value: actual value is false and predicted value is true (False positive)
  - bottom left value: actual value is true and predicted value is false (False positive)
  - bottom right value: actual value is true and predicted value is true (False positive)


This shows that the model is good at correctly predicting non-all nba players, but struggles at correctly identifying the other possibilities. To fix this class imbalance, one way would be to add weights to the model. This penalizes wrong predictions of the "all nba team" significantly more than wrong predictions of non "all nba team".

In [ ]:
def plot_logistic(model1, col):

  low = np.min(players_df[col])
  high = np.max(players_df[col])

  nonallnbaavg = np.mean(players_df[players_df['ALL-NBA'] == 0][col])
  allnbaavg = np.mean(players_df[players_df['ALL-NBA'] == 1][col])

  x = np.linspace(low, high, 200)
  x_prob = np.reshape(x, (200, 1))
  y_prob = model1.predict_proba(x_prob)[:,1]

  plt.plot(x_prob, y_prob, color = 'black')
  plt.vlines(nonallnbaavg, ymin = 0, ymax = 1, color='blue', linestyles='dashed', label= 'non-All NBA')
  plt.vlines(allnbaavg, ymin = 0, ymax = 1, color='red', linestyles='dashed', label='All NBA')
  plt.title('Probability of making All NBA team, given {}'.format(col))
  plt.xlabel(col)
  plt.legend(title = 'Mean', loc='lower right' )


In [ ]:
plot_logistic(logistic_regression('PTS', weight = 'balanced'), 'PTS')

In [ ]:
plot_logistic(logistic_regression('STL', weight = 'balanced'), 'STL')

In [ ]:
plot_logistic(logistic_regression('FG%', weight = 'balanced'), 'FG%')

## Multivariate logistic regression

In [ ]:
def logistic_regression2(col, weight=None, scale=False):

  X_train, X_test, Y_train, Y_test = train_test_split(predictors, outcome, test_size = 0.3)

  if type(col) == str:
    X_test = X_test[col].values.reshape(-1, 1)
    X_train = X_train[col].values.reshape(-1, 1)
  else:
    X_test = X_test[col]
    X_train = X_train[col]
    if scale:
      scaler = MinMaxScaler()
      X_test = scaler.fit_transform(X_test)
      X_train = scaler.fit_transform(X_train)

  model = LogisticRegression(class_weight = weight, max_iter=5000)
  model.fit(X_train, Y_train)

  predict = model.predict(X_test)
  accuracy = accuracy_score(Y_test, predict)
  roc = roc_auc_score(Y_test, predict)


  print("Accuracy: {}".format(accuracy))
  print("AUC-ROC: {}".format(roc))

  return model

In [ ]:
col = ['GS', 'MP', 'FG', 'FGA', '2P', '2PA', 'FT', 'FTA', 'DRB', 'TRB', 'AST', 'STL', 'PTS', 'TOV']
model = logistic_regression2(col, weight='balanced')

In [ ]:
col2 = ['GS', 'MP', 'FGA', '2PA', 'FTA', 'TRB', 'AST', 'STL', 'PTS', 'TOV']
model2 = logistic_regression2(col2, weight='balanced')

## Random forests

In [ ]:
rf = RandomForestRegressor(random_state = 1)
rf.fit(X_train, Y_train)

In [ ]:
predict = rf.predict(X_test)
acc = 1 - mean_squared_error(Y_test, predict)
roc = roc_auc_score(Y_test, predict)

print("MSE: {}".format(acc))
print("AUC-ROC: {}".format(roc))

Create grid search to find the best possible combinations of hyperparameters.

In [ ]:
n_estimators = [100, 500, 1000]
max_features = ['auto', 'log']
max_depth = [10, 20, 30]
min_samples_split = [10, 50, 100]
min_samples_leaf = [5, 10, 50]

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf,
              max_features = max_features)

gridF = GridSearchCV(rf, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, Y_train)

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, max_features=20, min_samples_leaf=1, min_samples_split=2, random_state=2)
rf.fit(X_train, Y_train)


predict = rf.predict(X_test)
acc = 1 - mean_squared_error(Y_test, predict)
roc = roc_auc_score(Y_test, predict)

print("MSE: {}".format(acc))
print("AUC-ROC: {}".format(roc))
